In [1]:
%matplotlib inline
import scipy.io as sio
from scipy.linalg import *
from numpy import *
from math import sqrt
from time import time
from scipy.io import loadmat,savemat
from random import shuffle
from os import listdir
import matplotlib.pyplot as plt
from sklearn import tree
import csv,pickle
from sklearn.feature_selection import RFE
from paramRegressor import *

In [2]:
def Listdir(folder):
    List = []
    for item in listdir(folder):
        if '.DS' not in item:
            List += [item]
    return List

In [4]:
stress = []
odfval = []
odfs = []
for filename in Listdir('data1'):
    dataList = loadmat('data1/'+filename)['Data']
    for item in dataList:
        if len(item)<80:
            odfs += [item[:76]]
            odfval += [count_nonzero(item)]
            stress += [item[78]]
        else:
            odfs += [item[:76]]
            odfval += [item[76]]
            stress += [item[79]]

In [5]:
stiffness = sio.loadmat('constraints.mat')['stiffness']

In [6]:
trainTot = int(0.8*len(odfs))

trainX,testX = odfs[:trainTot],odfs[trainTot:]
trainY,testY = stress[:trainTot],stress[trainTot:]

In [7]:
def runRegressor(regType,X,y, isShuffle=True, num=10):

    estimator = getEstimator(regType)
    X = np.array(X)
    y = np.array(y)
    maeList, r2List = [],[]
    pred,actual = [],[]
    kf = KFold(len(X), n_folds=num,shuffle=isShuffle)
    for k, (train, test) in enumerate(kf):

        estimator.fit(X[train], y[train])
        yPred = estimator.predict(X[test])
    
        pred += list(yPred)
        actual += list(y[test])
        
        mse = mean_squared_error(y[test],yPred)
        mae = mean_absolute_error(y[test],yPred)
        maeList += [mae]
        r2 = r2_score(y[test],yPred)
        r2List += [r2]
        print("[fold "+str(k)+"], r2_score:"+str(r2)+" mse:"+str(mse)+" mae:"+str(mae))
        
    print("[average ], r2_score:"+str(mean(r2List))+" mae:"+str(mean(maeList)))

In [8]:
def getFreqForOdf(odf):
    stiffness_avg = stiffness.dot(odf)
    stiffness_avg = stiffness_avg.reshape(6,6)
    S = inv(stiffness_avg)
    Ex = 1/S[0,0] 
    Gxy= 1/S[5,5]
    bendingFreq = getw1b(Ex)
    torsionalFreq = getw1t(Gxy)
    return bendingFreq,torsionalFreq

In [10]:
odfs

[array([  2.90172304e-09,   5.68478185e-05,   1.54125711e-10,
          2.76466784e-06,   8.49870356e-01,   1.14754758e-09,
          6.70617817e-11,   3.07700540e-02,   1.01979267e-09,
          1.63125758e-08,   7.30472097e-02,   1.48853174e+01,
          4.71480481e+01,   4.63444982e-08,   1.55757711e-03,
          4.21740799e-08,   2.21032045e-08,   5.69800787e-01,
          3.12918178e+01,   5.45452152e-03,   6.79478097e-04,
          4.65203339e-06,   1.26892344e-04,   1.58040105e-01,
          1.85790452e-09,   9.16252207e-04,   6.23592742e-01,
          8.92672990e-11,   2.79504241e-01,   1.95575497e-02,
          3.95005138e-10,   1.65748322e+00,   7.03067496e-02,
          7.77971861e-10,   1.84181096e-09,   1.20377757e-06,
          1.53054659e-06,   8.02212256e+01,   1.04566013e-04,
          4.58679866e-02,   2.68101020e-02,   2.42300666e+00,
          1.42770637e-01,   6.10142933e-10,   2.47386776e-06,
          2.08335403e-08,   8.11998319e-01,   1.17670955e-01,
        

In [3]:
def loadData(name,path='.'):
    '''
    This loads a pickle file and returns the content which is a DICTIONARY object in our case.
    '''
    if ".pkl" in name:
            name = name.split(".pkl")[0]
    if "/" in name:
            name = name.split("/",1)[1]

    with open(path+"/"+name + '.pkl', 'rb') as f:
          return pickle.load(f)

def saveData(obj, name,path='.'):
    '''
    This saves a object into a pickle file. In our case, it is generally a DICTIONARY object.
    '''

    with open(path+"/"+name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

In [6]:
x = loadData('odfs_data2_lower.pkl','Data')
x[:10]

[array([   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
         174.87277466,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.     

In [7]:
upper1 = loadData('odfs_data1_upper.pkl','Data')


In [8]:
upper11 = loadData('odf1_data1_upper.pkl','Data')
upper12 = loadData('odf2_data1_upper.pkl','Data')
upper13 = loadData('odf3_data1_upper.pkl','Data')

In [9]:
len(upper11),len(upper12),len(upper13)

(7, 11, 39)

In [10]:
stress11 = loadData('stress1_data1_upper.pkl','Data')
stress12 = loadData('stress2_data1_upper.pkl','Data')
stress13 = loadData('stress3_data1_upper.pkl','Data')

In [22]:
upper11[0]

array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,   82.17106167,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.     